In [1]:
import pandas as pd

In [3]:
ddf = pd.read_csv('DDF.csv', sep='|')

In [7]:
ddf.head(0)

,monto,es_fraude,balance_prev_or,balance_prev_des,hora,tipo_binario


In [4]:
ddf.shape

(2770360, 6)

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [8]:
X = ddf[['monto','balance_prev_or','balance_prev_des', 'hora','tipo_binario']]
y = ddf[['es_fraude']]

Normalización de datos

In [10]:
import pandas as pd
from sklearn.preprocessing import RobustScaler

# Normalizar los datos
scaler = RobustScaler()  # Definir el modelo de normalización
X_train = pd.DataFrame(scaler.fit_transform(X_train))  # Entrenar y normalizar los datos de entrenamiento, ajustamos parametros
X_test = pd.DataFrame(scaler.transform(X_test))  # Normalizar los datos de prueba

# Comprobar el resultado de la normalización
result = pd.concat([X_train.median(), X_test.median(),
                    (X_train.quantile(0.75) - X_train.quantile(0.25)),
                    (X_test.quantile(0.75) - X_test.quantile(0.25))],
                   axis=1,
                   keys=["Train median", "Test median", "Train iqr", "Test iqr"])
print(result)

   Train median  Test median  Train iqr  Test iqr
0  6.505361e-17     0.001348        1.0  1.001862
1  0.000000e+00     0.000678        1.0  1.001420
2  0.000000e+00     0.001214        1.0  1.002493
3  0.000000e+00     0.000000        1.0  1.000000
4  0.000000e+00     0.000000        0.0  0.000000


Balanceo de datos

In [12]:
from imblearn.over_sampling import SMOTE
'''generar muestras sintéticas adicionales de la clase minoritaria para equilibrar la distribución de clases en
el conjunto de datos.'''
smote = SMOTE(sampling_strategy='minority')
'''actualiza Xe ycon las muestras sintéticas generadas para abordar el problema de desequilibrio de clases en el
conjunto de datos.'''
X, y = smote.fit_resample(X, y)

#Creación del modelo

In [13]:
from sklearn.model_selection import train_test_split

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3867011, 5)
(3867011, 1)
(1657291, 5)
(1657291, 1)


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
model = Sequential(
    [
    Dense(4, input_dim=X_train.shape[1], activation='selu'),
    Dense(8 , activation='selu'),
    Dense(4, activation='selu'),
    Dense(1, activation='sigmoid')
    ]
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Recall'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 4)                 24        
                                                                 
 dense_13 (Dense)            (None, 8)                 40        
                                                                 
 dense_14 (Dense)            (None, 4)                 36        
                                                                 
 dense_15 (Dense)            (None, 1)                 5         
                                                                 
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________


In [ ]:
'''model = Sequential(
    [
    Dense(4, input_dim=X_train.shape[1], activation='selu'),
    Dense(8 , activation='selu'),
    Dense(4, activation='selu'),
    Dense(1, activation='sigmoid')
    ]
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['PositivePredictiveValue'])

model.summary()'''

In [ ]:
# Entrenamos el modelo
history = model.fit(X_train, y_train, shuffle=True, epochs=10, batch_size=16)

prediction = model.predict(X_test)
prediction = np.round(prediction)

Epoch 1/10
241689/241689 [==============================] - 524s 2ms/step - loss: 0.0856 - recall: 0.9714
Epoch 2/10
241689/241689 [==============================] - 515s 2ms/step - loss: 0.0531 - recall: 0.9861
Epoch 3/10
241689/241689 [==============================] - 512s 2ms/step - loss: 0.0494 - recall: 0.9869
Epoch 4/10
241689/241689 [==============================] - 517s 2ms/step - loss: 0.0465 - recall: 0.9881
Epoch 5/10
241689/241689 [==============================] - 515s 2ms/step - loss: 0.0448 - recall: 0.9886
Epoch 6/10
109980/241689 [============>.................] - ETA: 5:02 - loss: 0.0442 - recall: 0.9888